# Deep Learning for Coders - Chapter 1 [Draft]
> An even more top-down approach of the chapter

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [Deep Learning for Coders, Jupyter]
- image: images/trained_model.png
- author: Nathaniel D'Amours

This post is highly inspired from *Deep Learning for Coders* {% cite howard2020deep %}.

## Deep Learning is for Everyone

In [5]:
#hide
from fastbook import *
from pathlib import Path

In [6]:
#hide
def save_gv(filename, data, is_graph=False):
    if not is_graph:
        graph_data = gv(data)
    else:
        graph_data = data
    my_graph = graphviz.Source(graph_data)
    my_graph.render(filename, format='png', directory="my_icons/dl_for_coders_01/", view=False)

In [7]:
#hide
data = """
digraph {
    subgraph cluster_ai {
        label="Artificial Intelligence";
        "Your calculator";
        subgraph cluster_ml {
            label="Machine Learning";
            "Linear Regression";
            subgraph cluster_backend {
                label="Neural network";
                "Convolutional neural network";
            }
        }
    }
}
"""
save_gv("ai_ml_nn", data, is_graph=True)

![](my_icons/dl_for_coders_01/ai_ml_nn.png "Fig.0 - AI vs ML vs NN")

## What is Machine Learning ?

### A Machine Learning Program

As we learned earlier, machine learning is the science to write programs that learn. Therefore, machine learning could allow you to recognize dogs and cats without telling the program all the characteristics of each one (which is tricky) since the program can learn these. This learning is possible through this model training loop :

In [8]:
#hide
data = '''ordering=in
          Model[shape=box3d width=1 height=0.7]
          Inputs [shape=none, width=0.5 height=0.7]
          Inputs->Model->Predictions;
          Predictions->Performance
          Performance->Model[constraint=false label=updates]'''
save_gv("training", data)

![](my_icons/dl_for_coders_01/training.png "Fig.1 - The training loop")

Let's break down this figure :
1. The model receives **inputs** which are the data (the images of dogs and cats).
2. The model ouputs **predictions** which looks like : "Dog" or "Cat".
3. The **performance** of the predictions is calculated.
4. The **model** is updated according to the performance in order to improve itself.

### Architecture and Parameters

In [9]:
#hide
data = '''ordering=in
          Architecture[shape=box3d width=1 height=0.7]
          # Inputs [shape=none,label="" width=0.5 height=0.7]
          Inputs->Architecture->Predictions; Parameters->Architecture; Predictions->Performance
          Performance->Parameters[constraint=false label=updates]'''
save_gv("training_arch_param", data)

![](my_icons/dl_for_coders_01/training_arch_param.png "Fig.2 - Split the model into architecture and parameters")

You can notice that we split *Model* into *Architecture* and *Parameters*. The architecture is the functional form of the model and the parameters are some variables that defines how the architecture operates. For example, $y = ax + b$ is an architecture with the parameters $a$ and $b$ that change the behavior of the function.

### Labels and Loss

In [10]:
#hide
data = '''ordering=in
          Model[shape=box3d width=1 height=0.7 label=Architecture]
          Inputs->Model->Predictions; Parameters->Model;Labels->Loss; Predictions->Loss
          Loss->Parameters[constraint=false label=updates]'''
save_gv("training_labels_loss", data)

![](my_icons/dl_for_coders_01/training_labels_loss.png "Fig.3 - Split the performance into labels and loss")

We can see that now the *Performance* is split into *Labels* and *Loss*. The labels are the (ground) truth. For example, if an image is a dog the label of the image is *Dog*. Therefore, the labels and the predictions can be compared to determine the performance of the model. Indeed, if the prediction on an image is *Cat* and the label is *Dog*, you would know that the model did bad. The loss is this measure of performance thats compares the labels and the predictions so that we can updates the parameters to perform better.

### Trained Models

In [11]:
#hide
data = '''Model[shape=box3d width=1 height=0.7]
          Inputs->Model->Predictions'''
save_gv("trained_model", data)

![](my_icons/dl_for_coders_01/trained_model.png "Fig.4 - A trained model")

Once a model is trained. You can treat it as a regular program.

### Regular Programming

In [12]:
#hide
data = '''Program[shape=box3d width=1 height=0.7]
          Inputs->Program->Results'''
save_gv("regular_program", data)

![](my_icons/dl_for_coders_01/regular_program.png "Fig.5 - A regular program")

In [13]:
def add(a, b):
    return a + b

add(2, 3)

5

As you can see this program takes some inputs and outputs results. Indeed, the inputs are $2$ and $3$ and the result, $5$.

## What is Deep Neural Network ?

As we learned earlier deep neural network is a kind of machine learning model and "deep" refers to having move than 1 hidden layer (1 input layer → 1+ hidden layer → 1 output layer). This model can solve any problems according to the [*universal approximation theorem*](https://en.wikipedia.org/wiki/Universal_approximation_theorem) by varying the parameters. Therefore, we need a general "mecanism" to modify these parameters for each problem. This "mecanism" already exists and it is called stochastic gradient descent (SGD).

> Tip: SGD and deep neural networks sounds complex, but they aren't !

## How Our Image Recognizer Works

Let's break down the first lines of code of our image recognizer :

In [14]:
from fastai.vision.all import *

This allows us to use all the tools we will need to code a variety of computer vision models.

In [15]:
PATH = untar_data(URLs.PETS)/'images'
PATH

Path('C:/Users/natha/.fastai/data/oxford-iiit-pet/images')

This line downloads a dataset from fast.ai datasets collection (if not previously downloaded), extracts it (if not previously extracted) and returns a Path object with the extracted location

In [16]:
def is_cat(x):
    return x[0].isupper()

Here we define the function `is_cat` to get the label of an image. Indeed, the function returns `True` if the image contains a cat since the dataset's creators set cats's filenames with an upper case at the beginning.

In [17]:
#hide_output
dls = ImageDataLoaders.from_name_func(path=PATH,
                                      fnames=get_image_files(PATH),
                                      valid_pct=0.2,
                                      seed=42,
                                      label_func=is_cat,
                                      item_tfms=Resize(224))

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


Our model needs to know the kind and the structure of the dataset it's working with. Therefore, we created a dataloader. Since we are using images, this is an `ImageDataLoaders`. Also, `from_name_func` is used, because we are using the name of the files to label our images.

Let's explain the parameters :
- `path` : where the data is stored
- `fnames` : an object containing the Path objects of the images' filenames
- `valid_pct` : the percentage of data hold out randomly in the validation set (we will talk later about this)
- `seed` : aims to make your code reproductible by always generating the same validation set
- `label_func` : the function use to get the label of the image
- `item_tfms` : a transformation done to each item (in this case each item is resized to 224-pixel square)

{% bibliography --cited %}